In [14]:
import pandas as pd
import numpy as np
import random

def create_data(n, type):
    # job 갯수
    num_jobs = n
    # 균등분포 데이터 만들기
    def generate_input_data_uniform(num_jobs):
        data = []
        for job_id in range(1, num_jobs + 1):
            while True:
                release_time = 0 # 출제시간
                processing_time = int(np.random.uniform(1, 15)) # 소요시간 # 최소1, 최대10 균등분포
                due_date = int(np.random.uniform(release_time + processing_time, 40)) # 제출기한
                if  processing_time < due_date: # 제약조건
                    break
            rate = random.randint(1, 3) # 성적 반영 비율
            data.append((job_id, release_time, processing_time, due_date, rate))
        return data
    # 정규분포 데이터 만들기
    def generate_input_data_normal(num_jobs):
        data = []
        for job_id in range(1, num_jobs + 1):
            while True:
                processing_time = int(max(0, np.random.normal(10, 1))) # 평균5, 표준편차2 정규분포
                release_time = 0
                #release_time = int(max(0, np.random.normal(1, processing_time)) )
                due_date = int(max(release_time+processing_time, np.random.normal(8, 2)))
                if release_time < processing_time and processing_time < due_date:
                    break
            rate = random.randint(1, 3)
            data.append((job_id, release_time, processing_time, due_date, rate))
        return data
    
    if type == 'uniform':
        output_data = generate_input_data_uniform(num_jobs) # 균일분포로 데이터 생성
    else:
        output_data = generate_input_data_normal(num_jobs) # 정규분포로 데이터 생성

    df = pd.DataFrame(output_data, columns=['job', '출제시간', '소요시간', '제출기한', '성적 반영비율'])
    df = df.set_index('job').transpose()
    df.columns = ['job' + str(i) for i in range(1, num_jobs + 1)]
    df.index = ['출제시간', '소요시간', '제출기한', '성적 반영비율']

    filename = f'{num_jobs}_job_{type} data.csv'
    df.to_csv(filename, index=True, encoding='utf-8-sig')

    print(f"{num_jobs}개 job {type} data 생성.")

    return num_jobs, type

In [16]:
num_job, type = create_data(100, 'uniform')

100개 job uniform data 생성.


In [25]:
import matplotlib.pyplot as plt
import numpy as np
import random

In [41]:
np.arange(5)

array([0, 1, 2, 3, 4])

In [54]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

objective_list = ['total_flowtime', 'makespan', 'tardy_job', 'total_tardiness', 'total_weighted_tardiness']

params = {
    'MUT': 0.15,  # 변이확률
    'END' : 0.7,  # 설정한 비율만큼 sequence가 수렴하면 탐색을 멈추게 하는 파라미터
    'POP_SIZE' : 100,  # population size 10 ~ 100
    'NUM_OFFSPRING' : 30, # 한 세대에 발생하는 자식 chromosome의 수
    'CHANGE' : 10, # 다음 세대로 가는 자식 교체 수
    'type' : objective_list[3], # 원하는 목적함수
    'num_job' : 100 # job 갯수
    }
# ------------------------------
# 같은 문제 job갯수 100개 total_tardiness (제한시간 : 15분)

class GA_scheduling():
    def __init__(self, parameters):
        self.params = {}
        for key, value in parameters.items():
            self.params[key] = value

    def get_fitness(self, sequence):
        flowtime = 0
        total_flowtime = 0
        makespan = 0
        tardiness = 0
        total_tardiness = 0
        tardy_job = 0
        total_weighted_tardiness = 0
        
        for i in sequence:
            job = df['job' + str(i)]
            flowtime += job['소요시간']
            total_flowtime += flowtime

            makespan = flowtime
            
            if flowtime - job['제출기한'] >= 0:
                tardiness = flowtime - job['제출기한']
                tardy_job += 1
                total_tardiness += tardiness
            else:
                tardiness = 0

            weighted_tardiness = job['성적 반영비율'] * tardiness
            total_weighted_tardiness += weighted_tardiness

        ob_list = {'total_flowtime':total_flowtime, 'makespan':makespan, 'tardy_job':tardy_job,'total_tardiness':total_tardiness, 'total_weighted_tardiness':total_weighted_tardiness}
        return ob_list[self.params['type']]
    
    def print_average_fitness(self, population):
        population_average_fitness = 0
        for i in range(len(population)):
            population_average_fitness += population[i][1]
        population_average_fitness = population_average_fitness / len(population)
        #print("population 평균 fitness: {}".format(population_average_fitness))
        return population_average_fitness
    
    def sort_population(self, population):
        population.sort(key=lambda x:x[1], reverse = False)
        return population

    def selection_operater(self, population):
        # 토너먼트 선택(t보다 작으면 두 염색체 중 품질이 좋은 것을 선택)
        parents_list = []
        t = 0.6
        for i in range(2):
            sample = random.sample(population, 2)
            sample = self.sort_population(sample)
            rand = random.uniform(0,1)
            if rand < t :
                parents_list.append(sample[0][0])
            else:
                parents_list.append(sample[1][0])
        return parents_list[0], parents_list[1]

    def crossover_operater(self, mom_cho, dad_cho):
        # 순서 교차
        mom_ch = list(mom_cho)
        dad_ch = list(dad_cho)
        offspring_cho = []
        k1, k2 = sorted(random.sample(range(len(mom_ch)), 2)) # 0~9 중 2개 선택
        if k1 == 0:
            offspring_cho.extend(mom_ch[k1:k2])
            offspring_cho.extend([x for x in dad_ch[k2:] + dad_ch[:k2] if x not in mom_ch[k1:k2]])
        else:
            offspring_cho.extend([x for x in dad_ch[k2:] + dad_ch[:k2] if x not in mom_ch[k1:k2]][:k1])
            offspring_cho.extend(mom_ch[k1:k2])
            offspring_cho.extend([x for x in dad_ch[k2:] + dad_ch[:k2] if x not in offspring_cho])
        return offspring_cho

    def mutation_operater(self, chromosome):
        # exchange mutation
        # ex_mu1, ex_mu2 = sorted(random.sample(range(self.params['num_job']), 2)) # 0~9 중 2개 선택
        # chromosome[ex_mu1],chromosome[ex_mu2] = chromosome[ex_mu2], chromosome[ex_mu1]

        # scramble mutation
        sc_mu1, sc_mu2 = sorted(random.sample(range(self.params['num_job']), 2)) # 0~9 중 2개 선택
        scramble_list = chromosome[sc_mu1:sc_mu2]
        random.shuffle(scramble_list)
        chromosome[sc_mu1:sc_mu2] = scramble_list
        return chromosome

    def replacement_operator(self, population, offsprings):
        result_population = []
        population = self.sort_population(population)
        # 자식해 집단 중 뽑고 싶은 자식 수를 파라미터로 받아 가장 안좋은 해 대체
        offsprings = random.sample(offsprings, self.params["CHANGE"])
        for i in range(len(offsprings)):
            population[-(i+1)] = offsprings[i]
        result_population = self.sort_population(population)
        return result_population
    
    # 해 탐색(GA) 함수
    def search(self):
        generation = 0  # 현재 세대 수
        population = [] # 해집단
        offsprings = [] # 자식해집단
        average = []

        count = 1
        count_avg = []

        # 1. 초기화: 랜덤하게 해를 초기화
        for i in range(self.params["POP_SIZE"]):
            chromosome = list(range(1, self.params['num_job']+1))
            random.shuffle(chromosome)
            fitness = self.get_fitness(chromosome)
            population.append([chromosome, fitness])
            population = self.sort_population(population)
        print(f"minimize {self.params['type']} initialzed population : \n", population, "\n\n")

        while 1:
            offsprings = []
            for i in range(self.params["NUM_OFFSPRING"]):
                # 2. 선택 연산
                mom_ch, dad_ch = self.selection_operater(population)
                
                # 3. 교차 연산
                offspring = self.crossover_operater(mom_ch, dad_ch)

                # 4. 변이 연산
                # todo: 변이 연산여부를 결정, self.params["MUT"]에 따라 변이가 결정되지 않으면 변이연산 수행하지 않음
                if random.uniform(0,1) <= self.params["MUT"]:
                    offspring = self.mutation_operater(offspring)
                fitness = self.get_fitness(offspring)
                offsprings.append([offspring,fitness])
                
            # 5. 대치 연산
            population = self.replacement_operator(population, offsprings)
            generation += 1

            # self.print_average_fitness(population) # population의 평균 fitness를 출력함으로써 수렴하는 모습을 보기 위한 기능
            #average.append(self.print_average_fitness(population)) # population의 평균 fitness 그래프를 그리기 위한 average에 추가
            #average.append(population[-1][1])
            average.append(population[0][1])

            # 6. 알고리즘 종료 조건 판단
            same = 0
            for i in range(self.params["POP_SIZE"]):
                if population[0][1] == population[i][1]:
                    same += 1
            if same >= len(population) * self.params["END"]: # END비율만큼 수렴하면 정지
                plt.plot(np.arange(len(count_avg)), count_avg)
                plt.ylim(average[-1], average[0])
                plt.show()
                print("탐색이 완료되었습니다. \t 최종 세대수: {},\t 최종 해: {},\t 최종 적합도: {}".format(generation, population[0][0], population[0][1]))
                print('최종 population :', population)
                break

            count += 1

        # 최종적으로 얼마나 소요되었는지의 세대수, 수렴된 chromosome과 fitness를 출력
            # print("탐색이 완료되었습니다. \t 최종 세대수: {},\t 최종 해: {},\t 최종 적합도: {}".format(generation, population[0][0], population[0][1]))
            # print('최종 population :', population)
            # population의 평균 fitness 그래프
            if count%100 == 0:
                self.print_average_fitness(population) # population의 평균 fitness를 출력함으로써 수렴하는 모습을 보기 위한 기능
                count_avg.append(population[0][1])
                plt.ion()
                figure, ax = plt.subplot(figsize=(10,8))
                ax.plot(np.arange(len(count_avg)), count_avg)
                figure.canvas.draw()
                figure.canvas.flush_events()
                
                #plt.plot(np.arange(len(count_avg)), count_avg)
                #plt.ylim(average[-1], average[0])
                #plt.show(block = False)
                #plt.pause(0.5)
                #plt.close()

if __name__ == "__main__":
    input_data = pd.read_csv('100_job_uniform data.csv', index_col=0)
    df = input_data
    ga = GA_scheduling(params)
    ga.search()

minimize total_tardiness initialzed population : 
 [[[92, 72, 32, 2, 77, 41, 96, 49, 45, 58, 73, 86, 71, 70, 99, 29, 38, 44, 17, 56, 28, 48, 50, 80, 18, 15, 22, 52, 61, 13, 66, 39, 47, 12, 83, 84, 9, 24, 63, 60, 69, 59, 62, 6, 89, 94, 4, 35, 1, 34, 87, 11, 10, 27, 79, 30, 21, 91, 26, 16, 95, 19, 78, 98, 42, 31, 85, 51, 53, 55, 54, 90, 23, 93, 75, 76, 97, 37, 5, 14, 82, 36, 40, 20, 64, 46, 43, 67, 68, 74, 3, 33, 7, 8, 25, 65, 57, 81, 100, 88], 35782], [[92, 19, 100, 83, 57, 81, 74, 30, 56, 24, 87, 86, 47, 27, 90, 85, 40, 69, 73, 77, 18, 72, 33, 21, 22, 7, 2, 58, 68, 32, 61, 13, 6, 80, 70, 52, 20, 36, 28, 15, 45, 97, 89, 9, 60, 41, 71, 34, 42, 14, 59, 75, 78, 49, 99, 23, 51, 88, 8, 16, 84, 67, 96, 64, 79, 55, 3, 43, 53, 62, 95, 46, 31, 37, 35, 66, 39, 82, 26, 48, 25, 12, 65, 91, 63, 1, 29, 50, 98, 44, 4, 54, 38, 11, 93, 76, 5, 10, 17, 94], 36197], [[65, 46, 86, 19, 73, 99, 53, 92, 39, 10, 64, 34, 28, 43, 89, 72, 41, 40, 31, 22, 80, 69, 91, 14, 82, 7, 95, 13, 2, 74, 20, 16, 81, 47, 67, 27

AttributeError: 'AxesSubplot' object has no property 'figsize'

<Figure size 640x480 with 0 Axes>

In [8]:
#output_file = f'{num_job}_job_{type} data.csv'
output_file = '3_job_normal data.csv'
input_data = pd.read_csv(output_file, index_col=0)
df = input_data

def get_fitness(sequence):
    flowtime = 0
    total_flowtime = 0
    makespan = 0
    tardiness = 0
    total_tardiness = 0
    tardy_job = 0
    total_weighted_tardiness = 0
    
    for i in sequence:
        job = df['job' + str(i)]
        flowtime += job['출제시간'] + job['소요시간']
        total_flowtime += flowtime

        makespan = flowtime
        
        if flowtime - job['제출기한'] >= 0:
            tardiness = flowtime - job['제출기한']
            tardy_job += 1
            total_tardiness += tardiness
        else:
            tardiness = 0

        weighted_tardiness = job['성적 반영비율'] * tardiness
        total_weighted_tardiness += weighted_tardiness

    return total_flowtime, makespan, tardy_job, total_tardiness, total_weighted_tardiness

results = get_fitness([1,2,3])

print(f'total flowtime : {results[0]}')
print(f'makespan : {results[1]}')
print(f'number of tardy jobs : {results[2]}')
print(f'total_tardiness : {results[3]}')
print(f'total_weighted_tardiness : {results[4]}')

total flowtime : 31
makespan : 15
number of tardy jobs : 1
total_tardiness : 5
total_weighted_tardiness : 15


In [3]:
import itertools

#output_file = f'{num_job}_job_{type} data.csv'
output_file = '100_job_normal data.csv'
input_data = pd.read_csv(output_file, index_col=0)
df = input_data

def full_enumeration(object):
    num_jobs = len(df.columns)
    best_sequence = None
    best_objective = float('inf')
    best_set = []

    # 모든 가능한 순열 조합 생성
    permutations = list(itertools.permutations(range(1, num_jobs + 1), num_jobs))
    print(f'full_enumerate 갯수 {num_jobs}! :', len(permutations))
    
    for sequence in permutations:
        if object == 'total_flowtime':
            results = get_fitness(sequence)
            result = results[0]
        elif object == 'makespan':
            results = get_fitness(sequence)
            result = results[1]
        elif object == 'number of tardy jobs':
            results = get_fitness(sequence)
            result = results[2]
        elif object == 'total_tardiness':
            results = get_fitness(sequence)
            result = results[3]
        elif object == 'total_weighted_tardiness':
            results = get_fitness(sequence)
            result = results[4]

        objective = result

        if objective < best_objective:
            best_objective = objective
            best_sequence = sequence
            best_set = [[best_sequence, best_objective]]
            
        elif objective == best_objective and sequence != best_sequence:
            best_sequence = sequence
            best_set.append([best_sequence, best_objective])

    return object, best_sequence, best_objective, best_set


object, best_sequence, best_objective, best_set = full_enumeration('total_tardiness')

print(f'{object} 최소화')
print(f'Best sequence: {best_sequence}')
print(f'Best {object}: {best_objective}')
print(f'{object} 최소화 가능한 모든 sequence', best_set)